In [1]:
import zipfile
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.models import Model
import sys
import copy
import requests
from io import BytesIO

In [2]:
def extract_zip(zip_path, dest_dir, overwrite=False):
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"ZIP file not found: {zip_path}")

    # Ensure destination directory exists
    os.makedirs(dest_dir, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        if overwrite:
            # Remove existing files in destination if they exist
            for file in zip_ref.namelist():
                file_path = os.path.join(dest_dir, file)
                if os.path.exists(file_path):
                    if os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                    else:
                        os.remove(file_path)
        zip_ref.extractall(dest_dir)
        print(f"Extracted '{zip_path}' to '{dest_dir}'")

def extract_images_label_by_folder(zip_path, dest_dir, image_size=(224, 224),overwrite=False, zip_subdirectory = "",batch_size=32):
    extract_zip(zip_path, dest_dir, overwrite=False)
    train_dataset = tf.keras.utils.image_dataset_from_directory(
            dest_dir + "/dataset",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode='rgb',
            image_size=(224, 224),
            interpolation="nearest",
            batch_size=batch_size,
            validation_split=0.2,
            subset="training",
            seed=123)

    test_dataset = tf.keras.utils.image_dataset_from_directory(
            dest_dir + "/dataset",
            labels="inferred",
            label_mode="int",
            class_names=None,
            interpolation="nearest",
            color_mode='rgb',
            image_size=(224, 224),
            batch_size=batch_size,
            validation_split=0.2,
            subset="validation",
            seed=123)
    return train_dataset,test_dataset
def get_class_distribution(dataset):
    class_counts = {}
    for label in range(len(dataset.class_names)):
        class_counts[label] = 0
    for image, label in dataset.unbatch():
        class_counts[label.numpy()]+=1
    return class_counts

def plot_class_distribution(class_counts):
    sorted_class_counts = dict(sorted(class_counts.items(), key=lambda item: class_counts[item[0]], reverse=True))
    labels = list(sorted_class_counts.keys())
    values = list(sorted_class_counts.values())
    plt.figure(figsize=(15, 6))  # Adjust figure size for better readability
    plt.bar(labels, values)
    plt.xlabel("Class Labels")
    plt.ylabel("Frequency")
    plt.title("Class Distribution")
    plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility if needed
    plt.tight_layout() # Adjust layout to prevent labels from overlapping
    plt.show()

def balance_images_distribution(dataset, class_counts, plot_dist = False, top_p_to_avg = 0.1):

    def get_avg_entry_count(top_p_to_avg = 0.1):
        sorted_counts = dict(sorted(class_counts.items(), key=lambda item: item[1], reverse=True))
        values = list(sorted_counts.values())

        top_percent_index = int(len(values) * top_p_to_avg)
        top_percent_values = values[:top_percent_index]
        average_top_p = np.mean(top_percent_values)
        print(f"Average number of values for top p% entries: {average_top_p}")
        return average_top_p

    def augment(image_orig):
        image = tf.image.random_flip_left_right(image_orig)
        image = tf.image.random_brightness(image, max_delta=0.1)
        image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
        return image

    # Function to add an entry
    def add_entry(images, labels, image, label):
        images.append(image)
        labels.append(label)

    new_images = []
    new_labels = []
    avg_entry_count = get_avg_entry_count(top_p_to_avg)

    while(get_avg_entry_count(0.9) + 1 < avg_entry_count):
        for images, labels in dataset.unbatch():
            if(class_counts[labels.numpy()] < avg_entry_count):
                add_entry(new_images, new_labels,augment(images),labels)
                class_counts[labels.numpy()] += 1
    if(plot_dist):
        plot_class_distribution(class_counts)
    return tf.data.Dataset.from_tensor_slices((new_images, new_labels))

def one_hot_encode(image, label):
  return image, tf.one_hot(label, depth=len(classes))

def filter_p_percent(class_dist, p = 0.2):
    return (sorted(class_dist.keys(), key = lambda x: class_dist[x], reverse=True))[0:int(len(class_dist.keys())*p)];

In [4]:
zip_file_path = r"G:\Datasets\Pokemon151to10k.zip"
destination_directory = r"G:\Temp" #Replace with your temporary directory
unencoded_train_dataset,unencoded_test_dataset = extract_images_label_by_folder(zip_file_path, destination_directory, overwrite=False,batch_size=32)
classes = unencoded_train_dataset.class_names
class_dist = get_class_distribution(unencoded_train_dataset)

Extracted 'G:\Datasets\Pokemon151to10k.zip' to 'G:\Temp'
Found 10658 files belonging to 149 classes.
Using 8527 files for training.
Found 10658 files belonging to 149 classes.
Using 2131 files for validation.


In [7]:
train_dataset = unencoded_train_dataset.map(one_hot_encode)
test_dataset = unencoded_test_dataset.map(one_hot_encode)
# del unencoded_train_dataset
# del unencoded_test_dataset

In [15]:
train_dataset.element_spec

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.uint8, name=None),
 TensorSpec(shape=(None, 149), dtype=tf.float32, name=None))

In [8]:
# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) # Adjust input_shape if needed
# Freeze all layers in the base model
for layer in base_model.layers:
  layer.trainable = False
# Add custom classification layers
x = base_model.output
x = Flatten()(x)
x = BatchNormalization()(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

predictions = Dense(149, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with a slow learning rate
optimizer = keras.optimizers.Adam(learning_rate=1e-4) # Example: A very slow learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
267/267 [==============================] - ETA: 0s - loss: 3.0070 - accuracy: 0.4060   

In [ ]:
for layer in base_model.layers[-4:]:  # Unfreezing last 4 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train again with some layers unfrozen
model.fit(train_dataset, epochs=10, validation_data=test_dataset)